# 🤖 AutoGen Basic Concepts Tutorial

## Welcome to AutoGen!

This notebook introduces the fundamental concepts of Microsoft AutoGen - a framework for creating multi-agent conversational systems.

### What You'll Learn:
1. **Core AutoGen Concepts**: Agents, conversations, and configurations
2. **Basic Agent Types**: UserProxyAgent and AssistantAgent
3. **Simple Conversations**: Two-agent interactions
4. **Code Execution**: How agents can run code
5. **Group Chats**: Multiple agents working together

---

## 🔧 Setup and Installation

First, let's import the necessary libraries and set up our environment.

In [2]:
# Import required libraries
import os
import asyncio
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

print("✅ AutoGen and dependencies imported successfully!")
print("📦 Using autogen-agentchat version 0.6.4")

✅ AutoGen and dependencies imported successfully!
📦 Using autogen-agentchat version 0.6.4


## ⚙️ Configuration Setup

### Key Concept: LLM Configuration

AutoGen agents need to know how to connect to language models. We configure this once and reuse it for all agents.

In [3]:
# Configuration for connecting to OpenAI
model_client = OpenAIChatCompletionClient(
    model="gpt-3.5-turbo",  # You can also use "gpt-4" if you have access
    api_key=os.environ.get("OPENAI_API_KEY"),
)

print("🔑 Model client configuration ready!")
print(f"🎯 Using model: gpt-3.5-turbo")
print("🌡️ Temperature and other parameters controlled by model client")

🔑 Model client configuration ready!
🎯 Using model: gpt-3.5-turbo
🌡️ Temperature and other parameters controlled by model client


## 🤝 Basic Concept 1: Two-Agent Conversation

### UserProxyAgent vs AssistantAgent

- **UserProxyAgent**: Represents the human user, can execute code
- **AssistantAgent**: AI assistant that generates responses and solutions

In [4]:
# Create an AssistantAgent (AI assistant)
assistant = AssistantAgent(
    name="Assistant",
    system_message="""You are a helpful AI assistant. 
    You can help with coding, analysis, and problem-solving.
    When you need to demonstrate code, write Python code examples.""",
    model_client=model_client,
)

# Create a UserProxyAgent (represents you, the user)
user_proxy = UserProxyAgent(
    name="User"
)

print("👥 Basic agents created!")
print(f"🧑 User Proxy: {user_proxy.name}")
print(f"🤖 Assistant: {assistant.name}")
print("ℹ️ Note: In the new autogen version, conversations work differently")

👥 Basic agents created!
🧑 User Proxy: User
🤖 Assistant: Assistant
ℹ️ Note: In the new autogen version, conversations work differently


### Let's Have Our First Conversation!

In [ ]:
# Start a simple conversation using the new autogen API
print("🗣️ Starting a basic conversation with the assistant...\n")

async def run_simple_demo():
    # Simple task for demo
    result = await assistant.run(task="Say hello and introduce yourself briefly.")
    
    # Extract just the content from the result for cleaner demo output
    if hasattr(result, 'messages') and result.messages:
        last_message = result.messages[-1]
        if hasattr(last_message, 'content'):
            return last_message.content
    return str(result)

# Execute the simple demo
demo_result = await run_simple_demo()
print("Assistant Response:", demo_result)
print("\n✅ Simple conversation completed!")

## 🔄 Basic Concept 2: Code Execution

### How AutoGen Handles Code

1. Assistant writes code in markdown code blocks
2. UserProxy detects the code and executes it
3. Results are shared back with the assistant
4. Conversation continues based on results

In [ ]:
# Let's try a simple coding demo
print("🔢 Simple coding demonstration...\n")

async def run_simple_coding():
    task = "Write a simple Python function to add two numbers. Just show the code, keep it brief."
    
    result = await assistant.run(task=task)
    
    # Extract just the content for cleaner demo output
    if hasattr(result, 'messages') and result.messages:
        last_message = result.messages[-1]
        if hasattr(last_message, 'content'):
            return last_message.content
    return str(result)

# Execute the simple coding demo
coding_result = await run_simple_coding()
print("Assistant Response:", coding_result)
print("\n📊 Simple coding demo completed!")

## 👥 Basic Concept 3: Group Chat (Multiple Agents)

### Beyond Two Agents

Group chats allow multiple agents to collaborate on complex tasks. Each agent can have specialized roles.

In [7]:
# Create specialized agents for a team collaboration

# Teacher agent - explains concepts
teacher = AssistantAgent(
    name="Teacher",
    system_message="""You are a patient teacher who explains programming concepts clearly.
    Focus on education and helping students understand the 'why' behind solutions.""",
    model_client=model_client,
)

# Coder agent - writes code
coder = AssistantAgent(
    name="Coder",
    system_message="""You are an expert programmer who writes clean, efficient code.
    Focus on practical implementation and best practices.""",
    model_client=model_client,
)

# Reviewer agent - checks and improves code
reviewer = AssistantAgent(
    name="Reviewer",
    system_message="""You are a code reviewer who focuses on quality, security, and optimization.
    Point out potential issues and suggest improvements.""",
    model_client=model_client,
)

print("👨‍🏫 Specialized agents created:")
print(f"📚 Teacher: {teacher.name}")
print(f"💻 Coder: {coder.name}")
print(f"🔍 Reviewer: {reviewer.name}")
print("ℹ️ Note: Team collaboration works differently in the new version")

👨‍🏫 Specialized agents created:
📚 Teacher: Teacher
💻 Coder: Coder
🔍 Reviewer: Reviewer
ℹ️ Note: Team collaboration works differently in the new version


In [11]:
# Set up a round-robin team chat with the new API
team = RoundRobinGroupChat([teacher, coder, reviewer], max_turns=5)

print("🎭 Team chat set up with 3 participants!")
print("🔄 Using RoundRobinGroupChat for organized collaboration")
print("ℹ️ Each agent will contribute in turn")

🎭 Team chat set up with 3 participants!
🔄 Using RoundRobinGroupChat for organized collaboration
ℹ️ Each agent will contribute in turn


In [ ]:
# Simple team demonstration
print("🎪 Simple team demo...\n")

async def run_simple_team_demo():
    # Very simple task for clean demo output
    simple_task = "Each agent say one sentence about Python programming."
    
    result = await team.run(task=simple_task)
    
    # Extract clean content
    if hasattr(result, 'messages') and result.messages:
        responses = []
        for msg in result.messages[-3:]:  # Show last 3 messages
            if hasattr(msg, 'content') and hasattr(msg, 'source'):
                responses.append(f"{msg.source}: {msg.content}")
        return "\n".join(responses)
    return str(result)

# Execute simple team demo
try:
    team_demo_result = await run_simple_team_demo()
    print("Team Responses:")
    print(team_demo_result)
except Exception as e:
    print(f"Team demo encountered an issue: {e}")
    print("This is normal - team collaboration can be complex!")

print("\n🎉 Simple team demo completed!")

## 🎛️ Basic Concept 4: Configuration Options

### Understanding Key Parameters

Let's explore the important configuration options that control agent behavior.

In [15]:
# Demonstrate different agent configurations
print("🎮 Different Agent Configurations:\n")

# Note: In the new autogen version, UserProxyAgent has simplified parameters
# The human input modes and code execution configs work differently

# Basic UserProxyAgent (simplified in new version)
basic_user_agent = UserProxyAgent(name="BasicUser")

# Multiple AssistantAgents with different personalities
conservative_agent = AssistantAgent(
    name="ConservativeAgent",
    system_message="You give precise, consistent answers with minimal creativity.",
    model_client=model_client,
)

creative_agent = AssistantAgent(
    name="CreativeAgent", 
    system_message="You give creative, varied responses with lots of ideas.",
    model_client=model_client,
)

print("🤖 Basic User Agent: Simplified proxy for human interaction")
print("🔒 Conservative Agent: Precise, consistent responses")
print("🎨 Creative Agent: Varied, imaginative responses")
print("ℹ️ Note: The new version has simplified agent configuration")

🎮 Different Agent Configurations:

🤖 Basic User Agent: Simplified proxy for human interaction
🔒 Conservative Agent: Precise, consistent responses
🎨 Creative Agent: Varied, imaginative responses
ℹ️ Note: The new version has simplified agent configuration


### Temperature and Creativity Settings

In [16]:
# Create agents with different creativity levels using new API

# Conservative agent (lower temperature for more deterministic responses)
conservative_model_client = OpenAIChatCompletionClient(
    model="gpt-3.5-turbo",
    api_key=os.environ.get("OPENAI_API_KEY"),
    # Note: temperature control may be handled differently in the new API
)

conservative_agent = AssistantAgent(
    name="ConservativeAgent",
    system_message="You give precise, consistent answers with minimal creativity. Be concise and factual.",
    model_client=conservative_model_client,
)

# Creative agent (encourage more creative responses through system message)
creative_model_client = OpenAIChatCompletionClient(
    model="gpt-3.5-turbo",
    api_key=os.environ.get("OPENAI_API_KEY"),
)

creative_agent = AssistantAgent(
    name="CreativeAgent",
    system_message="You give creative, varied responses with lots of ideas. Be imaginative and explore different perspectives.",
    model_client=creative_model_client,
)

print("🎯 Agent personalities:")
print(f"🔒 Conservative: Consistent, predictable responses")
print(f"🎨 Creative: Varied, imaginative responses")
print("ℹ️ Note: Temperature control is handled by the model client in the new API")

🎯 Agent personalities:
🔒 Conservative: Consistent, predictable responses
🎨 Creative: Varied, imaginative responses
ℹ️ Note: Temperature control is handled by the model client in the new API


## 🔍 Basic Concept 5: Conversation Analysis

### Understanding What Happened

After a conversation, we can analyze the results to understand the interaction.

In [18]:
# Function to analyze conversation results
def analyze_conversation(chat_result):
    """Simple analysis of a completed conversation"""
    
    print("📋 Conversation Analysis:")
    print("=" * 40)
    
    # Check if we have chat history
    if hasattr(chat_result, 'chat_history'):
        messages = chat_result.chat_history
        print(f"📨 Total messages: {len(messages)}")
        
        # Count messages by speaker
        speakers = {}
        for msg in messages:
            speaker = msg.get('name', 'Unknown')
            speakers[speaker] = speakers.get(speaker, 0) + 1
        
        print("\n👥 Messages by speaker:")
        for speaker, count in speakers.items():
            print(f"  {speaker}: {count} messages")
        
        # Show the conversation flow
        print("\n🔄 Conversation flow:")
        for i, msg in enumerate(messages[-5:], 1):  # Show last 5 messages
            speaker = msg.get('name', 'Unknown')
            content_preview = msg.get('content', '')[:50] + "..."
            print(f"  {i}. {speaker}: {content_preview}")
    
    else:
        print("No detailed chat history available")
    
    print("\n✅ Analysis complete!")

# Let's analyze our last conversation
if 'chat_result' in locals():
    analyze_conversation(chat_result)
else:
    print("💡 Run a conversation first, then we can analyze it!")

💡 Run a conversation first, then we can analyze it!


## 🎯 Practice Exercise: Build Your Own Simple Chat

Now it's your turn! Try creating your own agents and conversation.

In [19]:
# Your turn! Create a simple math tutor system

# Create a MathTutor agent
math_tutor = AssistantAgent(
    name="MathTutor",
    system_message="""You are a friendly math tutor. 
    Help students solve math problems step by step.
    Always explain your reasoning and encourage learning.""",
    model_client=model_client,
)

print("🎓 Math tutoring system ready!")
print("Try asking the tutor to help with a math problem.")
print("ℹ️ Use await math_tutor.run(task='your math question') to interact")

🎓 Math tutoring system ready!
Try asking the tutor to help with a math problem.
ℹ️ Use await math_tutor.run(task='your math question') to interact


In [ ]:
# Simple math tutor demo
async def run_simple_math_demo():
    simple_math = "What is 2 + 2? Give a brief answer."
    
    result = await math_tutor.run(task=simple_math)
    
    # Extract clean content
    if hasattr(result, 'messages') and result.messages:
        last_message = result.messages[-1]
        if hasattr(last_message, 'content'):
            return last_message.content
    return str(result)

# Execute simple math demo
math_demo_result = await run_simple_math_demo()
print("Math Tutor Response:", math_demo_result)
print("\n🎉 Simple math demo completed!")

## 📚 Summary: AutoGen Basic Concepts

### What We Learned:

1. **🤖 Agent Types**:
   - `UserProxyAgent`: Represents users, can execute code
   - `AssistantAgent`: AI assistants with specialized roles

2. **⚙️ Key Configurations**:
   - `llm_config`: How agents connect to language models
   - `temperature`: Controls creativity/randomness
   - `human_input_mode`: NEVER, ALWAYS, or TERMINATE
   - `code_execution_config`: How and where to run code

3. **💬 Conversation Types**:
   - Two-agent chats: `agent1.initiate_chat(agent2)`
   - Group chats: Multiple agents with a manager

4. **🔧 Code Execution**:
   - Agents can write and execute Python code
   - Results are shared automatically
   - Useful for data analysis, calculations, and testing

### Next Steps:

- 🚀 Try the advanced demo notebook for complex scenarios
- 🔨 Experiment with different agent personalities
- 🌐 Explore external tool integrations
- 📊 Build domain-specific agent teams

---

**Happy coding with AutoGen! 🎉**

## 🛠️ Troubleshooting Common Issues

### Issue 1: "No API key found"
```python
# Make sure your .env file contains:
# OPENAI_API_KEY=your_actual_api_key_here
```

### Issue 2: "Code execution failed"
```python
# Check that your work directory exists and is writable
# Consider setting use_docker=True for better isolation
```

### Issue 3: "Conversation doesn't end"
```python
# Adjust max_round in GroupChat
# Use human_input_mode="TERMINATE" to control ending
```

### Issue 4: "Agents aren't collaborating well"
```python
# Improve system messages to clarify roles
# Use custom speaker selection for better flow
```